# Initialization and package preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Data load and preprocessing

In [15]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
y_train

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.2, random_state=rso)

X_train, y_train = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)
X_dev, y_dev = X_train[:10000,:], y_train[:10000]

# Load back neural network and give it a try

In [7]:
with open('search_lr_wd.pkl', 'rb') as f:
    neural_outcome = pickle.load(f)

In [22]:
import torch 
from torch.autograd import Variable
dtype = torch.FloatTensor


nn_model = neural_outcome['best_net']
nn_model.net.eval()
# X_train_test_tensor = torch.from_numpy(X_train_test.values)
# X_train_test_tensor= X_train_test_tensor.type(dtype)
# out = nn_model.test(X_train_test_tensor)
# nn_model.check_auc(out,y_train_test)


idx = test_data['id']
test_data_mutated = test_data.drop(['id'], axis=1)
test_data_mutated = naive_pre.dtype_change(test_data_mutated)
test_data_mutated = naive_pre.scale(test_data_mutated, test=True)
test_data_tensor = torch.from_numpy(test_data_mutated.values)
test_data_tensor = test_data_tensor.type(dtype)
out = nn_model.test(test_data_tensor)

In [44]:
result= np.hstack((idx.values.reshape(892816,1), out.data.numpy()[:,1].reshape(892816,1)))
print(result.shape)
result[:,0] = result[:,0].astype(np.int32)
np.savetxt('submission.csv',result, fmt=['%d', '%.9f'] , delimiter= ',', header= 'id, target')

(892816, 2)


In [ ]:
# ! cat submission.csv 10
#! sudo pip3 install lightgbm

# Try to use Lightgbm here.

In [2]:
! svn export https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide

/bin/sh: 1: svn: not found


In [79]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
y_train

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.2, random_state=rso)

X_train, y_train, col = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)
X_dev, y_dev = X_train[:10000,:], y_train[:10000]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [106]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train, free_raw_data=False)

lgb1_params = {
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric': {'binary_logloss','l2'},
    'learning_rate': 0.01, 
    'feature_fraction': 0.9,   #
    'bagging_fraction': 0.8,   #
    'bagging_freq': 5,
    'verbose': 0
}

# lgb1 = lgb.train(lgb1_params,
#         lgb_train,
#         num_boost_round =1250,  #Number of trees it will generate  
#         valid_sets= lgb_val,
#     early_stopping_rounds = 10
#         )

# lgb1 = lgb.cv(lgb1_params,
#         lgb_train,
#         num_boost_round =50,  #Number of trees it will generate  
#     early_stopping_rounds = 10
#         )

lgb1_hyper = {'params': lgb1_params,
             'train_set': lgb_train,
              'valid_sets': lgb_val,
              'num_boost_round': 1500, 
              'early_stopping_rounds': 5
             }

lgb1=  lgb.train(**lgb1_hyper)

# lgb2_params = {
#     'boosting_type':'gbdt',
#     'objective':'binary',
#     'metric': 'binary_logloss', 
#     'feature_fraction': 0.8,
#     'bagging_fraction': 0.6,
#     'bagging_freq': 5,
#     'feature_freq' :10,
#     'num_leaves':16  
# }

# lgb2 = lgb.train(lgb2_params,
#         lgb_train,
#         valid_sets= lgb_val,
#     num_boost_round = 2500,
#     learning_rates= lambda x: 0.05*(0.999**x), #step_decay
#     early_stopping_rounds = 5
#         )

# lgb3_params = {
#     'boosting_type':'gbdt',
#     'objective':'binary',
#     'metric': 'binary_logloss',
#     'learning_rate': 0.02,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.6,
#     'bagging_freq': 5,
#     'max_depth':4
# }

# lgb3 = lgb.train(lgb3_params,
#         lgb_train,
#         valid_sets= lgb_val,
#     num_boost_round = 800,
#     early_stopping_rounds = 10
#         )

y_pred1 = lgb1.predict(X_train_test)
# y_pred2 = lgb2.predict(X_train_test)
# y_pred3 = lgb3.predict(X_train_test)



# lgb_params_1 = {
#     'learning_rate': 0.01,
#     'n_estimators': 1250,
#     'max_bin': 10,
#     'subsample': 0.8,
#     'subsample_freq': 10,
#     'colsample_bytree': 0.8,   #feature_fraction
#     'min_child_samples': 500
# }

# lgb_params_2 = {
#     'learning_rate': 0.005,
#     'n_estimators': 3700,
#     'subsample': 0.7,
#     'subsample_freq': 2,
#     'colsample_bytree': 0.3,  
#     'num_leaves': 16
# }

# lgb_params_3 = {
#     'learning_rate': 0.02,
#     'n_estimators': 800,
#     'max_depth': 4
# }

[1]	valid_0's l2: 0.246799	valid_0's binary_logloss: 0.686745
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.24363	valid_0's binary_logloss: 0.680406
[3]	valid_0's l2: 0.240516	valid_0's binary_logloss: 0.674176
[4]	valid_0's l2: 0.237404	valid_0's binary_logloss: 0.667949
[5]	valid_0's l2: 0.234399	valid_0's binary_logloss: 0.661933
[6]	valid_0's l2: 0.231465	valid_0's binary_logloss: 0.656056
[7]	valid_0's l2: 0.228536	valid_0's binary_logloss: 0.650187
[8]	valid_0's l2: 0.225672	valid_0's binary_logloss: 0.644444
[9]	valid_0's l2: 0.222816	valid_0's binary_logloss: 0.638712


KeyboardInterrupt: 

In [103]:
from sklearn import metrics


# k = (y_pred1+y_pred2+y_pred3)/3
# metrics.roc_auc_score(y_train_test,k)
# lgb1.feature_importance()
# col
# metrics.roc_
#lgb1.best_score
#metrics.roc_auc_score(y_train_test,y_pred1)
# k =np.array([col, lgb1.feature_importance()])
# print(k.transpose(1,0))
#len(lgb1.feature_importance())
lgb1

{'l2-mean': [0.24651600868537682,
  0.24309109790517258,
  0.23975574723195675,
  0.23646861007919368,
  0.2332301683138252,
  0.23007437220946536,
  0.22695960988636146,
  0.22392481210311654,
  0.22092545677567027,
  0.2179267742168709,
  0.21504192923127249,
  0.21220600074119078,
  0.20943530793530202,
  0.206681023458886,
  0.20400604244603163,
  0.20132405163202316,
  0.19868456374534677,
  0.19604481926397738,
  0.19360407608956071,
  0.19110245509158655,
  0.18871590109391634,
  0.18616597081162137,
  0.18375737442623302,
  0.18138854849317732,
  0.17908725300492173,
  0.1768436665137687,
  0.17469708281907886,
  0.1724973785522374,
  0.17032470122968799,
  0.16821074709969336,
  0.16614725544577122,
  0.16407805157447447,
  0.16210553759486696,
  0.16009560013762197,
  0.15822224249573275,
  0.15627900654107502,
  0.15437404329675006,
  0.15254663287842646,
  0.15070005280720361,
  0.14887751480618233,
  0.14708125051793938,
  0.14534341428261413,
  0.14355175276401663,
  0.14

In [ ]:

train_data11 = data_util.load_train_data()
train_data11.head(10)
# for col in train_data11.columns:
#     if 'bin' in col or 'cat' in col:
#         train_data11[col] = train_data11[col].astype('category')
# data = pd.get_dummies(train_data11)
# data.columns

uselessones = ['ps_ind_14',
             'ps_ind_10_bin',
             'ps_ind_11_bin',
             'ps_ind_12_bin', 
             'ps_ind_13_bin',
             'ps_ind_18_bin',
             'ps_car_10_cat',
             'ps_calc_15_bin',
             'ps_calc_20_bin'
        ]
print(train_data11.columns)
train_data11 = train_data11.drop(uselessones, axis =1)
print('hhhhh')
print(train_data11.columns)

# If increase data will boost my performance??

In [91]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
y_train

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.3, random_state=rso)

X_train, y_train, col = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)
X_dev, y_dev = X_train[:10000,:], y_train[:10000]


import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train, free_raw_data=False)

lgb1_params = {
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.01, 
    'feature_fraction': 0.9,   #
    'bagging_fraction': 0.8,   #
    'bagging_freq': 5,
    'verbose': 0
}

lgb1 = lgb.train(lgb1_params,
        lgb_train,
        num_boost_round =1250,  #Number of trees it will generate  
        valid_sets= lgb_val,
    early_stopping_rounds = 5
        )

y_pred1 = lgb1.predict(X_train_test)

from sklearn import metrics


metrics.roc_auc_score(y_train_test,y_pred1)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


[1]	valid_0's binary_logloss: 0.686745
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.680406
[3]	valid_0's binary_logloss: 0.674176
[4]	valid_0's binary_logloss: 0.667949
[5]	valid_0's binary_logloss: 0.661933
[6]	valid_0's binary_logloss: 0.656056
[7]	valid_0's binary_logloss: 0.650187
[8]	valid_0's binary_logloss: 0.644444
[9]	valid_0's binary_logloss: 0.638712
[10]	valid_0's binary_logloss: 0.633112
[11]	valid_0's binary_logloss: 0.627682
[12]	valid_0's binary_logloss: 0.622309
[13]	valid_0's binary_logloss: 0.616939
[14]	valid_0's binary_logloss: 0.611778
[15]	valid_0's binary_logloss: 0.606761
[16]	valid_0's binary_logloss: 0.601612
[17]	valid_0's binary_logloss: 0.596555
[18]	valid_0's binary_logloss: 0.591561
[19]	valid_0's binary_logloss: 0.58662
[20]	valid_0's binary_logloss: 0.581778
[21]	valid_0's binary_logloss: 0.577143
[22]	valid_0's binary_logloss: 0.572236
[23]	valid_0's binary_logloss: 0.567589
[24]	valid_0's binary_logloss

0.63933418607869774